In [2]:
# We will use of our own algorithms like KNN to label the images
# We have stored faces of different people in our dataset
# Now we will bring one of those people in front of camera and our algo will tell the name of the person

#Steps 
#Load the dataset
#We will take all the files stored in the folder
#Then we will store names and label it in our dictionary
#All the images and their labels will be stored and concatenated in a training set
#Start the streaming
#Only focus on face
#Then pass the face and training set together to detect the face

#1. Import the useful libraries
import numpy as np
import cv2
import os

#Copied the KNN code earlier done by me
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2)) #We will find squares of distance between the two matrices
    #Then we will sum the squares

# Test Time 
def knn(X,Y,k=5):
    #Writing KNN
    vals = []
    m = X.shape[0]
    #Traversing all rows
    for i in range(m):
        ix = X[i,:-1] #Features in the dataset
        iy = X[i,-1] #Label of that
        d = dist(Y,ix) #Distance between current features and those stored
        #Here we will find 
        vals.append((d,iy)) #Putting labels and distances
        #Updating array with the distance and it's value
    
    vals = sorted(vals)
    # Nearest/First K points
    vals = vals[:k]
    
    vals = np.array(vals)
    #Converting into numpy
    #print(vals)
    
    new_vals = np.unique(vals[:,1],return_counts=True) #This array will store all the unique values in the second column 
    #Then it will also frequency of the numbers
    
    #print(new_vals)
    
    index = new_vals[1].argmax()
    #Here we found the index of number which came most number of times
    pred = new_vals[0][index]
    #Using that index we find the number which came most number of times
    return pred #Will return the label of the image
#######

#Camera 
cap = cv2.VideoCapture(0)
classifier = cv2.CascadeClassifier("Haarcascade.xml")

#2. Set the path and took array where we will store the photograohs
skip = 0 #To count every 10th
dataset_path = './data/' #Will store in this folder
face_data = []
labels=[]

class_id = 0 #Labels for the file
names = {} #Will store the name of the persons associated with the labels

#Now we need to load the training data and do the data preparation
for fx in os.listdir(dataset_path): #Checking every file in the given folder
    if fx.endswith('.npy'): #Checked all the files we needed
        #Create a mapping btw class_id and name
        names[class_id] = fx[:-4] #Last four characters are.npy so we remove them and get the name
        print("Loaded "+fx) #Simply write the name of the folders
        data = np.load(dataset_path+fx) #This will load all the images of the persons in different lists
        face_data.append(data) #Now face_data contains all the images of the different people
        
        #Create Labels for the class
        target = class_id*np.ones((data.shape[0],)) #First it will contain matrrix full of zeroes, then ones, and so on
        class_id += 1
        labels.append(target) #Now all the labels are loaded, I have taken three different people so labels would be 0,1, and 2

face_dataset = np.concatenate(face_data,axis=0) 
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)
#In KNN algorithm the trainset contains both sets of features and the labels associated with them , hence we need to concatenate
#them together

print(trainset.shape)

# Testing 

while True:
    ret,img = cap.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #Converted it to gray because the saved images are in this colour
     
    if ret == False:
        continue

    faces = classifier.detectMultiScale(img,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face

    #Get the face ROI (Only need face)
        offset = 10
        face_section = gray[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100)) 

        #Predicted Label (out)
        out = knn(trainset,face_section.flatten()) #Will send face and compare it with all images in the dataset

        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)] #Will save the name according to the label
        cv2.putText(gray,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA) #The name will be shown on screen
        cv2.rectangle(gray,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",gray)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('a'):
        break

cap.release()
cv2.destroyAllWindows()

Loaded Pawan.npy
Loaded Sachin.npy
Loaded Sangeeta.npy
(32, 10000)
(32, 1)
(32, 10001)
